##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tf.function

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/eager/tf_function"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/beta/tutorials/eager/tf_function.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/beta/tutorials/eager/tf_function.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>

텐서플로 2.0에서는 즉시 실행(eager execution)이 기본적으로 실행됩니다. 이는 매우 직관적이고 유연한 유저 인터페이스를 제공합니다.(일회성 연산을 실행하는 게 훨씬 쉽고 빠릅니다) 그러나 어느 정도 성능과 전개성(배포 가능성)을 희생시켜야 가능할 수 있습니다.  
In TensorFlow 2.0 eager execution is turned on by default. This gets you a very
intuitive and flexible user interface (running one-off operations is much easier
and faster) but this can come at the expense of performance and deployability.

최상의 성능을 얻고 모델을 어디에서나 배포 가능하게 할 수 있도록, 프로그램의 그래프를 만들 수 있는 도구로 `tf.funtion`을 제공합니다.
To get peak performance and to make your model deployable anywhere, we provide
`tf.function` as the tool you can use to make graphs out of your programs.
오토그래프 덕분에, 놀라운 양의 파이썬 코드가 tf.function과 함께 간단히 작동됩니다. 하지만 여전히 몇 가지 주의해야 할 점들이 있습니다.
Thanks to AutoGraph, a surprising amount of Python code just works with
tf.function, but there are still pitfalls to be wary of.

주요 요점 및 권장사항:
The main takeaways and recommendations are:

- 객체 번형이나 리스트 추가와 같은 파이썬 부수 효과에 의존하지 마세요.
- tf.function은 넘파이 연산이나 파이썬 기본 자료형 보다는 텐서플로 연산과 가장 잘 작동합니다.
- 의구심이 생길경우, `for x in y` 구문이 작동할 것입니다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tf-nightly-2.0-preview
import tensorflow as tf

  ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview


In [2]:
import contextlib

# 발생할 수 있는 오류를 보여주는 몇가지 헬퍼 코드입니다.
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('예상된 예외 \n  {}: {}'.format(error_class, e))
  except Exception as e:
    print('예상되지 않은 예외 \n  {}: {}'.format(type(e), e))
  else:
    raise Exception('{}가 발생할 것으로 예상되나, 오류가 발생하지 않았습니다!'.format(
        error_class))

정의한 `tf.function`은 마치 텐서플로 연산과 같습니다. 즉각적으로 실행할 수 있고, 그래프로 사용할 수 있고, 그래디언트 등을 가지고 있습니다.

In [ ]:
# 연산 같은 함수

@tf.function
def add(a, b):
  return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

In [ ]:
# 그래디언트를 포함한 함수

@tf.function
def add(a, b):
  return a + b

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

In [ ]:
# 함수 속에 함수들을 사용할 수 있습니다.

@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

## 추적과 다형성 Tracing and Polymorphism

파이썬의 동적 타이핑은 다양한 타입의 매개변수로 함수를 호출할 수 있다는 것을 의미합니다. 파이썬은 각 시나리오마다 다른 것을 수행할 것입니다. 반면에, 텐서플로 그래프는 정적 데이터 타입과 형태의 차원이 필요합니다. `tf.function`은 올바른 그래프를 생성할 필요가 있을 때 함수를 되돌아가 이 둘 사이의 차이를 연결합니다. `tf.function` 사용법의 중요 세부 요소들은 대부분 이렇게 돌아가는 과정에서 생겨납니다. 

어떠한 상황이 일어났는지 보기 위해 다른 타입의 매개변수를 사용하여 함수를 호출할 수 있습니다.

On the other hand, TensorFlow graphs require static dtypes and shape dimensions. `tf.function` bridges this gap by retracing the function when necessary to generate the correct graphs. Most of the subtlety of `tf.function` usage stems from this retracing behavior.

You can call a function with arguments of different types to see what is happening.

In [3]:
# 함수의 다형성

@tf.function
def double(a):
  print("추적", a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()


W0606 17:59:10.771329 28908 tf_logging.py:161] Entity <function double at 0x0000021B76048048> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte (tmpbl86aj26.py, line 5)


추적 Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)



W0606 17:59:10.982763 28908 tf_logging.py:161] Entity <function double at 0x0000021B76048048> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte (tmpmiquo6dv.py, line 5)
W0606 17:59:11.052576 28908 tf_logging.py:161] Entity <function double at 0x0000021B76048048> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte (tmp7g5m2dns.py, line 5)


추적 Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

추적 Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



추적 동향을 제어하기 위해서는 다음 기술들을 참고하세요.
To control the tracing behavior, use the following techniques:

- 새로운 `tf.function` 생성: 분리된 `tf.function` 객체는 추적이 공유되지 않게 보장됩니다. or 추적을 공유할 수 없게 
- `get_concrete_function` 특정한 추적을 얻기 위한 메소드 사용
- Specify `input_signature` when calling `tf.function` to ensure only one function graph will be built.
- 하나의 function 그래프기 빌드되기위해 `tf.function`을 호출할때 특정한 `input_signatur` 사용

In [4]:
print("구체적인 추적 얻기")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print("추적된 함수 실행")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))
print("적절하지 않는 종류의 추적을 사용할 경우 오류가 발생할 수 있습니다.")
with assert_raises(tf.errors.InvalidArgumentError):
  double_strings(tf.constant(1))

W0606 17:59:59.622725 28908 tf_logging.py:161] Entity <function double at 0x0000021B76048048> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte (tmpg3o8lgq2.py, line 5)


구체적인 추적 얻기
추적 Tensor("a:0", dtype=string)
추적된 함수 실행
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
적절하지 않는 종류의 추적을 사용할 경우 오류가 발생할 수 있습니다.
예상된 예외 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_33 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_33]


In [5]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
  print("추적", x)
  return tf.where(tf.equal(x % 2, 0), x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# 입력 시그니쳐에 1-D 텐서를 주었으므로, 오류가 발생할 것입니다.
with assert_raises(ValueError):
  next_collatz(tf.constant([[1, 2], [3, 4]]))


W0606 18:00:43.988104 28908 tf_logging.py:161] Entity <function next_collatz at 0x0000021B770936A8> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte (tmpnjw8q_v6.py, line 5)


추적 Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
예상된 예외 
  <class 'ValueError'>: Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=60, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name=None),))


## When to retrace?
다형성을 가지고 있는 `tf.function`은 추적과정에서 생성된 구체적인 함수들의 캐시를 유지합니다. 캐시의 키들은 함수의 매개변수와 주요 매개변수에서 생성된 효과적인 키의 튜플입니다. 키는 
A polymorphic `tf.function` keeps a cache of concrete functions generated by tracing. The cache keys are effectively tuples of keys generated from the function args and kwargs. `tf.Tensor`에 대해 생성된 키는 `tf.Tensor`의 형태와 타입입니다. 파이썬에 대해 생성된 키는 파이썬 프리메티브의 값을 나타냅니다. The key generated for a `tf.Tensor` argument is its shape and type. The key generated for a Python primitive is its value. For all other Python types, the keys are based on the object `id()` so that methods are traced independently for each instance of a class. 앞으로, 텐서플로 더욱 정교한 캐쉬를 추가할 것입니다. In the future, TensorFlow may add more sophisticated caching for Python objects that can be safely converted to tensors.

## 파이썬 또는 텐서 매개변수?

그래프의 구조와 하이퍼파라미터(hyperparameter)를 제어하기위해 파이썬 매개변수가 자주 사용됩니다. 예를 들면 `num_layers=10` 또는 `training=True` 또는 `nonlinearity='relu'` 등이 있습니다. 그래서 만약 파이썬 매개변수가 변경된다면 그래프를 다시 추적하는 것이 이치에 맞습니다. 

그러나, 파이썬 매개변수가 그래프 구조를 제어하는데 사용되지 않을 수도 있습니다. 이럴 경우, 파이썬 값을 변경하는 것은 불필요한 추적을 발생시킬 수 있습니다. 예를 들면, 오토 그래프가 동적으로 전개하는 아래 셀의 훈련 루프를 살펴보면, 다중 추적에도 불구하고, 생성된 그래프는 사실상 동일하므로 다소 비효율적입니다. However, it's possible that a Python argument is not being used to control graph construction. In these cases, a change in the Python value can trigger needless retracing. Take, for example, this training loop, which AutoGraph will dynamically unroll. Despite the multiple traces, the generated graph is actually identical, so this is a bit inefficient.

In [6]:
def train_one_step():
  pass

@tf.function
def train(num_steps):
  print("num_steps 추적 = {}".format(num_steps))
  for _ in tf.range(num_steps):
    train_one_step()

train(num_steps=10)
train(num_steps=20)


W0606 18:18:44.085334 28908 tf_logging.py:161] Entity <function train at 0x0000021B770ED510> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: (unicode error) 'utf-8' codec can't decode byte 0xc3 in position 10: invalid continuation byte (tmpivq5sbzp.py, line 3)


num_steps 추적 = 10


TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

The simple workaround here is to cast your arguments to Tensors if they do not affect the shape of the generated graph.

In [ ]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

## Side effects in `tf.function`

In general, Python side effects (like printing or mutating objects) only happen during tracing. So how can you reliably trigger side effects from `tf.function`?

The general rule of thumb is to only use Python side effects to debug your traces. Otherwise, TensorFlow ops like `tf.Variable.assign`, `tf.print`, and `tf.summary` are the best way to ensure your code will be traced and executed by the TensorFlow runtime with each call. In general using a functional style will yield the best results. 

In [ ]:
@tf.function
def f(x):
  print("추적", x)
  tf.print("실행", x)

f(1)
f(1)
f(2)


If you would like to execute Python code during each invocation of a `tf.function`, `tf.py_function` is an exit hatch. The drawback of `tf.py_function` is that it's not portable or particularly performant, nor does it work well in distributed (multi-GPU, TPU) setups. Also, since `tf.py_function` has to be wired into the graph, it casts all inputs/outputs to tensors.

In [ ]:
external_list = []

def side_effect(x):
  print('파이썬 부수 효과')
  external_list.append(x)

@tf.function
def f(x):
  tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# py_function이 1을 tf.constant(1)로 캐스팅하기 때문에 .numpy()를 호출할 필요가 있습니다.
assert external_list[0].numpy() == 1


## Beware of Python state

생성자(generators)나 반복자(iterators)와 같은 많은 파이썬 특성들은 상태를 추적하기 위해 파이썬 런타임에 의존합니다. Many Python features, such as generators and iterators, rely on the Python runtime to keep track of state. In general, while these constructs work as expected in Eager mode, many unexpected things can happen inside a `tf.function` due to tracing behavior.

한 가지 예를 들자면, 반복자의 상태를 전진시키는 것은 파이썬의 부수 효과이며, 그러므로 오직 추적과정에서 발생합니다. To give one example, advancing iterator state is a Python side effect and therefore only happens during tracing.

In [ ]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
  external_var.assign_add(next(iterator))
  tf.print("외부 변수의 값:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)


If an iterator is generated and consumed entirely within the tf.function, then it should work correctly. However, the entire iterator is probably being traced, which can lead to a giant graph. This may be what you want. But if you're training on an large in-memory dataset represented as a Python list, then this can generate a very large graph, and `tf.function` is unlikely to yield a speedup.

If you want to iterate over Python data, the safest way is to wrap it in a tf.data.Dataset and use the `for x in y` idiom. AutoGraph has special support for safely converting `for` loops when `y` is a tensor or tf.data.Dataset.


In [ ]:
def measure_graph_size(f, *args):
  g = f.get_concrete_function(*args).graph
  print("{}({}) contains {} nodes in its graph".format(
      f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

@tf.function
def train(dataset):
  loss = tf.constant(0)
  for x, y in dataset:
    loss += tf.abs(y - x) # Some dummy computation.
  return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))


When wrapping Python/Numpy data in a Dataset, be mindful of `tf.data.Dataset.from_generator` versus ` tf.data.Dataset.from_tensors`. The former will keep the data in Python and fetch it via `tf.py_function` which can have performance implications, whereas the latter will bundle a copy of the data as one large `tf.constant()` node in the graph, which can have memory implications.

Reading data from files via TFRecordDataset/CsvDataset/etc. is the most effective way to consume data, as then TensorFlow itself can manage the asynchronous loading and prefetching of data, without having to involve Python.

## Automatic Control Dependencies

A very appealing property of functions as the programming model, over a general dataflow graph, is that functions can give the runtime more information about what was the intended behavior of the code.

For example, when writing code which has multiple reads and writes to the same variables, a dataflow graph might not naturally encode the originally intended order of operations. In `tf.function`, we resolve ambiguities in execution order by referring to the execution order of statements in the original Python code. This way, ordering of stateful operations in a `tf.function` replicates the semantics of Eager mode.

This means there's no need to add manual control dependencies; `tf.function` is smart enough to add the minimal set of necessary and sufficient control dependencies for your code to run correctly.

In [ ]:
# Automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
  a.assign(y * b)
  b.assign_add(x * a)
  return a + b

f(1.0, 2.0)  # 10.0


## 변수

We can use the same idea of leveraging the intended execution order of the code to make variable creation and utilization very easy in `tf.function`. There is one very important caveat, though, which is that with variables it's possible to write code which behaves differently in eager mode and graph mode.

Specifically, this will happen when you create a new Variable with each call. Due to tracing semantics, `tf.function` will reuse the same variable each call, but eager mode will create a new variable with each call. To guard against this mistake, `tf.function` will raise an error if it detects dangerous variable creation behavior.

In [ ]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v

with assert_raises(ValueError):
  f(1.0)

In [ ]:
# Non-ambiguous code is ok though

v = tf.Variable(1.0)

@tf.function
def f(x):
  return v.assign_add(x)

print(f(1.0))  # 2.0
print(f(2.0))  # 4.0


In [ ]:
# 변수들이 함수가 실행될 때 처음으로 생성되었다는것을 증명할 수 있는한, tf.function안에 변수를 생성할 수 있습니다.
# You can also create variables inside a tf.function as long as we can prove that those variables are created only the first time the function is executed.

class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
  if obj.v is None:
    obj.v = tf.Variable(1.0)
  return obj.v.assign_add(x)

print(g(1.0))  # 2.0
print(g(2.0))  # 4.0

In [ ]:
# 변수 이니셜라이저는 함수의 매개변수와 다른 변수들의 값에 의존할 수 있습니다. 
# Variable initializers can depend on function arguments and on values of other
# variables. We can figure out the right initialization order using the same
# method we use to generate control dependencies.

state = []
@tf.function
def fn(x):
  if not state:
    state.append(tf.Variable(2.0 * x))
    state.append(tf.Variable(state[0] * 3.0))
  return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))


# Using AutoGraph

The [autograph](https://www.tensorflow.org/guide/autograph) library is fully integrated with `tf.function`, and it will rewrite conditionals and loops which depend on Tensors to run dynamically in the graph.

`tf.cond` and `tf.while_loop` continue to work with `tf.function`, but code with control flow is often easier to write and understand when written in imperative style.

In [ ]:
# Simple loop

@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([5]))

In [ ]:
# If you're curious you can inspect the code autograph generates.
# It feels like reading assembly language, though.

def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

print(tf.autograph.to_code(f))

## AutoGraph: Conditionals

AutoGraph will convert `if` statements into the equivalent `tf.cond` calls.

This substitution is made if the condition is a Tensor. Otherwise, the conditional is executed during tracing.

In [ ]:
def test_tf_cond(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'cond' for node in g.as_graph_def().node):
    print("{}({}) uses tf.cond.".format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print("{}({}) executes normally.".format(
        f.__name__, ', '.join(map(str, args))))


In [ ]:
@tf.function
def hyperparam_cond(x, training=True):
  if training:
    x = tf.nn.dropout(x, rate=0.5)
  return x

@tf.function
def maybe_tensor_cond(x):
  if x < 0:
    x = -x
  return x

test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))
test_tf_cond(maybe_tensor_cond, tf.constant(-1))
test_tf_cond(maybe_tensor_cond, -1)


`tf.cond` has a number of subtleties.
- it works by tracing both sides of the conditional, and then choosing the appropriate branch at runtime, depending on the condition. Tracing both sides can result in unexpected execution of Python code
- it requires that if one branch creates a tensor used downstream, the other branch must also create that tensor.


In [ ]:
@tf.function
def f():
  x = tf.constant(0)
  if tf.constant(True):
    x = x + 1
    print("Tracing `then` branch")
  else:
    x = x - 1
    print("Tracing `else` branch")
  return x

f()

In [ ]:
@tf.function
def f():
  if tf.constant(True):
    x = tf.ones([3, 3])
  return x

# Throws an error because both branches need to define `x`.
with assert_raises(ValueError):
  f()

## AutoGraph and loops

AutoGraph has a few simple rules for converting loops.

- `for`: Convert if the iterable is a tensor
- `while`: Convert if the while condition depends on a tensor

If a loop is converted, it will be dynamically unrolled with `tf.while_loop`, or in the special case of a `for x in tf.data.Dataset`, transformed into `tf.data.Dataset.reduce`.

If a loop is _not_ converted, it will be statically unrolled 

In [ ]:
def test_dynamically_unrolled(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'while' for node in g.as_graph_def().node):
    print("{}({}) uses tf.while_loop.".format(
        f.__name__, ', '.join(map(str, args))))
  elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
    print("{}({}) uses tf.data.Dataset.reduce.".format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print("{}({}) gets unrolled.".format(
        f.__name__, ', '.join(map(str, args))))


In [ ]:
@tf.function
def for_in_range():
  x = 0
  for i in range(5):
    x += i
  return x

@tf.function
def for_in_tfrange():
  x = tf.constant(0, dtype=tf.int32)
  for i in tf.range(5):
    x += i
  return x

@tf.function
def for_in_tfdataset():
  x = tf.constant(0, dtype=tf.int64)
  for i in tf.data.Dataset.range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_range)
test_dynamically_unrolled(for_in_tfrange)
test_dynamically_unrolled(for_in_tfdataset)


In [ ]:
@tf.function
def while_py_cond():
  x = 5
  while x > 0:
    x -= 1
  return x

@tf.function
def while_tf_cond():
  x = tf.constant(5)
  while x > 0:
    x -= 1
  return x

test_dynamically_unrolled(while_py_cond)
test_dynamically_unrolled(while_tf_cond)

 If you have a `break` or early `return` clause that depends on a tensor, the top-level condition or iterable should also be a tensor.

In [ ]:
@tf.function
def buggy_while_py_true_tf_break(x):
  while True:
    if tf.equal(x, 0):
      break
    x -= 1
  return x

@tf.function
def while_tf_true_tf_break(x):
  while tf.constant(True):
    if tf.equal(x, 0):
      break
    x -= 1
  return x

with assert_raises(TypeError):
  test_dynamically_unrolled(buggy_while_py_true_tf_break, 5)
test_dynamically_unrolled(while_tf_true_tf_break, 5)

@tf.function
def buggy_py_for_tf_break():
  x = 0
  for i in range(5):
    if tf.equal(i, 3):
      break
    x += i
  return x

@tf.function
def tf_for_tf_break():
  x = 0
  for i in tf.range(5):
    if tf.equal(i, 3):
      break
    x += i
  return x

with assert_raises(TypeError):
  test_dynamically_unrolled(buggy_py_for_tf_break)
test_dynamically_unrolled(tf_for_tf_break)




In order to accumulate results from a dynamically unrolled loop, you'll want to use `tf.TensorArray`.


In [ ]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
  return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
  # [batch, time, features] -> [time, batch, features]
  input_data = tf.transpose(input_data, [1, 0, 2])
  max_seq_len = input_data.shape[0]

  states = tf.TensorArray(tf.float32, size=max_seq_len)
  state = initial_state
  for i in tf.range(max_seq_len):
    state = rnn_step(input_data[i], state)
    states = states.write(i, state)
  return tf.transpose(states.stack(), [1, 0, 2])
  
dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

As with `tf.cond`, `tf.while_loop` also comes with a number of subtleties.
- Since a loop can execute 0 times, all tensors used downstream of the while_loop must be initialized above the loop
- The shape/dtypes of all loop variables must stay consistent with each iteration

In [ ]:
@tf.function
def buggy_loop_var_uninitialized():
  for i in tf.range(3):
    x = i
  return x

@tf.function
def f():
  x = tf.constant(0)
  for i in tf.range(3):
    x = i
  return x

with assert_raises(ValueError):
  buggy_loop_var_uninitialized()
f()

In [ ]:
@tf.function
def buggy_loop_type_changes():
  x = tf.constant(0, dtype=tf.float32)
  for i in tf.range(3): # Yields tensors of type tf.int32...
    x = i
  return x

with assert_raises(tf.errors.InvalidArgumentError):
  buggy_loop_type_changes()

In [ ]:
@tf.function
def buggy_concat():
  x = tf.ones([0, 10])
  for i in tf.range(5):
    x = tf.concat([x, tf.ones([1, 10])], axis=0)
  return x

with assert_raises(ValueError):
  buggy_concat()
  
@tf.function
def concat_with_padding():
  x = tf.zeros([5, 10])
  for i in tf.range(5):
    x = tf.concat([x[:i], tf.ones([1, 10]), tf.zeros([4-i, 10])], axis=0)
    x.set_shape([5, 10])
  return x

concat_with_padding()


## 다음 단계

이제 코드에 더욱 빨리 `tf.function`이전의 노트북들을 다시 방문하여 Now revisit the earlier notebooks and try using `tf.function` to speed up your code!